In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [34]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (132.05 seconds): Collecting URLs... yahoo               

100.00% (1.52 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,26078,26013,182
yahoo,410,410,40
reuters,1157,1157,24
dailymail,2884,2884,14
investing.com,925,925,13
...,...,...,...
news24,1214,1214,0
techcrunch,20,20,0
arstechnica,211,211,0
zdnet,327,327,0


,url,pubs,actual_pub
0,https://politico.com/news/2019/10/17/pelosi-de...,"politico, realclearpolitics",politico
1,https://politico.com/news/2019/10/16/elizabeth...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/10/17/trump-syr...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/15...,"politico, realclearpolitics",politico
4,https://politico.com/magazine/story/2019/10/17...,"politico, realclearpolitics",politico
...,...,...,...
60,https://nbcnews.com/news/us-news/overrated-gen...,"msnbc, nbcnews",nbcnews
61,https://msnbc.com/ali-velshi/watch/romney-aban...,"msnbc, nbcnews",msnbc
62,https://nbcnews.com/politics/politics-news/pen...,"msnbc, nbcnews",nbcnews
63,https://nbcnews.com/news/latino/house-holds-it...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [35]:
download_result = crawler.download(); download_result

14.29% (9.35 seconds): Downloading... abcnews             

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


35.16% (20.00 seconds): Downloading... arynews             

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


100.00% (153.47 seconds): Downloading... yahoo               

,downloaded,trashed,total
all,113,69,182
yahoo,32,8,40
reuters,13,11,24
dailymail,12,2,14
investing.com,5,8,13
...,...,...,...
news24,0,0,0
techcrunch,0,0,0
arstechnica,0,0,0
zdnet,0,0,0
